# Download The Dependencies or however you spell that. Dependencys, Dependencies

In [3]:
import tensorflow as tf
import numpy as np
from PIL import Image
import csv 
import mplfinance as mpf
import pandas as pd
import time
from matplotlib import pyplot as plt
import os

2022-11-10 07:36:16.435014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Hyper-parameters

In [4]:
NUM_CANDLES = 90

# Get the data from the CSV file

In [7]:
start_time = time.time()

df = pd.read_csv('/Users/spencerfonbuena/Desktop/ES.txt', header = None, names = ['Date', 'open', 'high', 'low', 
    'close', 'volume'], sep=',',index_col=0,parse_dates=True)


In [8]:
df['close']

Date
2005-09-06 16:30:00    1096.25
2005-09-06 16:35:00    1096.00
2005-09-06 16:40:00    1095.75
2005-09-06 16:45:00    1096.00
2005-09-06 16:50:00    1096.25
                        ...   
2022-08-12 16:35:00    4281.50
2022-08-12 16:40:00    4281.50
2022-08-12 16:45:00    4281.50
2022-08-12 16:50:00    4281.50
2022-08-12 16:55:00    4280.75
Name: close, Length: 1191027, dtype: float64

# Take the CSV data, and turn it into images

### This also takes the images and seperates them into different folders which represent each class we are trying to predict



In [5]:
#initialize number of candles to be shown on graph
A = 500000
B = A + NUM_CANDLES
C = 0
label_yhat = []
for i in range(0, 100):

    #store the number of candles you want shown on the graph
    storage = df.iloc[A:B]
    #create the graph
    mc = mpf.make_marketcolors(up='g',down='r')
    s  = mpf.make_mpf_style(marketcolors=mc)


    #find 1 percent and 2 percent above and below
    one_low = df['close'][B] * .99
    two_low = df['close'][B] * .98
    one_high = df['close'][B] * 1.01
    two_high = df['close'][B] * 1.02
    #initialize the label counter
    label_counter = B

    #this is to make sure that once it either enters the "gone up by one percent" or "gone down by 1 percent"
    #it doesn't enter the other while loops
    pathway = 0
    #look for the instance when the price increases or decreases by 1 percent
    while df['low'][label_counter] >= one_low and df['high'][label_counter] <= one_high:
        label_counter += 1

    #If the price moved up 1 pecent first, this while loop will trigger and check if it is a two to one, or a one to one trade
    while df['low'][label_counter] >= one_low and df['high'][label_counter] <= two_high:
        label_counter += 1
        pathway = 1
    #Check if price has increased two percent
    if df['high'][label_counter] >= two_high and pathway == 1:
        label_yhat.append('twoup')
    #check if price has reversed back down to the one percent marker
    elif df['low'][label_counter] <= one_low and pathway == 1:
        label_yhat.append('oneup')
    
    #if the price moved down 1 pecent first, this will check if it is a two to one, or a one to one trade
    while df['high'][label_counter] <= one_high and df['low'][label_counter] >= two_low and pathway != 1:
        label_counter += 1
        pathway = 2
    #check if the price has continued down two percent
    if df['low'][label_counter] <= two_low and pathway == 2:
        label_yhat.append('twodown')
    #check if price reversed back up to the 1 percent above marker
    elif df['high'][label_counter] >= one_high and pathway == 2:
        label_yhat.append('onedown')

    #converts the list of labels into a numpy array
    yhat = np.array(label_yhat)

    # seperates the photos into classifications
    if yhat[C] == 'twoup':
        file_path = '/Users/spencerfonbuena/Documents/Images/two_up/' + str(i) + '.png'
        mpf.plot(storage,type='candle',savefig=file_path , warn_too_much_data = 10000000, style = s, volume=True)
        #this resized the image to be a 512x512 resolution imgae
        sized_image = Image.open('/Users/spencerfonbuena/Documents/Images/two_up/' + str(i) + '.png').crop((177,40,689,552)).save('/Users/spencerfonbuena/Documents/Images/two_up/' + str(i) + '.png')
    elif yhat[C] == 'oneup':
        file_path = '/Users/spencerfonbuena/Documents/Images/one_up/' + str(i) + '.png'
        mpf.plot(storage,type='candle',savefig=file_path , warn_too_much_data = 10000000, style = s, volume=True)
        #this resized the image to be a 512x512 resolution imgae
        sized_image = Image.open('/Users/spencerfonbuena/Documents/Images/one_up/' + str(i) + '.png').crop((177,40,689,552)).save('/Users/spencerfonbuena/Documents/Images/one_up/' + str(i) + '.png')
    elif yhat[C] == 'onedown':
        file_path = '/Users/spencerfonbuena/Documents/Images/one_down/' + str(i) + '.png'
        mpf.plot(storage,type='candle',savefig=file_path , warn_too_much_data = 10000000, style = s, volume=True)
        #this resized the image to be a 512x512 resolution imgae
        sized_image = Image.open('/Users/spencerfonbuena/Documents/Images/one_down/' + str(i) + '.png').crop((177,40,689,552)).save('/Users/spencerfonbuena/Documents/Images/one_down/' + str(i) + '.png')
    elif yhat[C] == 'twodown':
        file_path = '/Users/spencerfonbuena/Documents/Images/two_down/' + str(i) + '.png'
        mpf.plot(storage,type='candle',savefig=file_path , warn_too_much_data = 10000000, style = s, volume=True)
        #this resized the image to be a 512x512 resolution imgae
        sized_image = Image.open('/Users/spencerfonbuena/Documents/Images/two_down/' + str(i) + '.png').crop((177,40,689,552)).save('/Users/spencerfonbuena/Documents/Images/two_down/' + str(i) + '.png')
        
    #increment the graph by one 15 minute interval 
    A += 1
    B += 1
    C += 1
        





# Use TS.Keras to label the data
## This will also load the data into our data pipeline

In [6]:
data = tf.keras.utils.image_dataset_from_directory('/Users/spencerfonbuena/Documents/Images', image_size=(512,512))

Found 100 files belonging to 4 classes.


2022-11-10 05:08:48.337117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Pre-Process the Data
I am fairly certain that this simply makes us able to access our data pipeline. It alos normalizes the data. Be careful however not to run this twice. If you do, it will divide your RGB pixel values by 255 twice, and then you will end up with a black screen

In [7]:
data = data.map(lambda x, y: (x/255, y))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

# Split the data into train, dev, and test sets

In [8]:
train_size = int(len(data) * .90) # 95 percent training set
dev_size = int(len(data) * .05) # 2.5 percent development set
test_size = int(len(data) * .05) # 2.5 percent test set

In [9]:
train = data.take(train_size)
development = data.skip(train_size).take(dev_size)
test = data.skip(train_size + dev_size).take(test_size)

# Initialize the Model

In [10]:
model = tf.keras.Sequential()

# Forward Propogation Model (More or Less) 
## VGG16 with 528 Million parameters

In [11]:
                                                                                                                                            #Output Shapes
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1 ,padding='same', activation='relu', input_shape=(512,512,3) )) #(512,512,64)
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(512,512,64)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)) #(256,256,64)

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(256,256,128)
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(256,256,128)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)) #(128,128,128)

model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(128,128, 256)
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(128,128,256)
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(128,128,256)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)) #(64,64,256)

model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(64,64,512)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)) #(32, 32, 512)

model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1 ,padding='same', activation='relu')) #(32,32,512)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)) #(16,16,512)

model.add(tf.keras.layers.Flatten()) #(0, 131072)
model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
model.add(tf.keras.layers.Dense(units=4, activation='softmax'))



# Back Propogation 

In [12]:
model.compile(optimizer='adam', loss= tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the Model

In [13]:
logdir = '/Users/spencerfonbuena/Documents/logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [14]:
hist = model.fit(train, epochs=10, validation_data=development, callbacks=[tensorboard_callback])

Epoch 1/10
2/3 [===================>..........] - ETA: 1:52 - loss: 0.6810 - accuracy: 0.8906